In [2]:
import pandas as pd
import json
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
import ast
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split

# PJZ Dataset

In [56]:
# 데이터 불러오기
pjz_df = pd.read_excel('/home/k-cat/TH/K-CAT/lth/data/PJZ_en_ko.xlsx')
pjz_df.head(10)

,source,label,message_idx,author,time,en,ko,time.1
0,PJ chats,1,0,decoy,0.611111,Hey Its Mads,"안녕, 나 매즈야","""14:40"""
1,PJ chats,1,0,Billy Joe,0.611111,Hey babes,"안녕, 자기야","""14:40"""
2,PJ chats,1,0,Billy Joe,0.611806,Almost done with work. So glad,거의 일 다 끝났어. 정말 기뻐,"""14:41"""
3,PJ chats,1,0,decoy,0.611806,Hey:) that's good right?,안녕 :) 그거 좋은 거지?,"""14:41"""
4,PJ chats,1,0,Billy Joe,0.612500,Yea,응,"""14:42"""
5,PJ chats,1,0,decoy,0.613194,So what's new lol,그럼 뭐 새로운 일 있어? ㅋㅋ,"""14:43"""
6,PJ chats,1,0,Billy Joe,0.613194,Nothing,없어,"""14:43"""
7,PJ chats,1,0,Billy Joe,0.613194,did u touch it in the shower. Lol,샤워할 때 그거 만졌어? ㅋㅋ,"""14:43"""
8,PJ chats,1,0,decoy,0.613889,No had to hurry up lol,"아니, 빨리 해야 했어 ㅋㅋ","""14:44"""
9,PJ chats,1,0,decoy,0.613889,Hold on a min,잠깐만,"""14:44"""


In [57]:
# 전처리
pjz_df = pjz_df[pjz_df['ko'] != 'na']
pjz_df = pjz_df[['source', 'label', 'message_idx', 'author', 'ko']]
pjz_df.rename(columns={'ko': 'text'}, inplace=True)
pjz_df = pjz_df.drop_duplicates(['text'], keep='first').reset_index(drop=True)
pjz_df

,source,label,message_idx,author,text
0,PJ chats,1,0,decoy,"안녕, 나 매즈야"
1,PJ chats,1,0,Billy Joe,"안녕, 자기야"
2,PJ chats,1,0,Billy Joe,거의 일 다 끝났어. 정말 기뻐
3,PJ chats,1,0,decoy,안녕 :) 그거 좋은 거지?
4,PJ chats,1,0,Billy Joe,응
...,...,...,...,...,...
7569,PJ chats,1,621,decoy,"나 곧 가야 해, 엄마가 소리치고 있어."
7570,PJ chats,1,621,jackjohnsons7,그거 좋았어?
7571,PJ chats,1,621,jackjohnsons7,그래서 너 하트포드 어디 거리에서 살아?
7572,PJ chats,1,621,decoy,너한테 안 알려줘.


In [58]:
# message_idx 별로 author 리스트로 저장 (딕셔너리)
author_dict = {}
for idx in pjz_df['message_idx'].unique():
    author_dict[idx] = set(pjz_df[pjz_df['message_idx'] == idx]['author'].tolist())


# author_dict의 value에 decoy가 포함되어 있으면 제거
for idx, authors in author_dict.items():
    if 'decoy' in authors:
        author_dict[idx].remove('decoy')
        
author_dict

{np.int64(0): {'Billy Joe'},
 np.int64(1): {'Billy Joe'},
 np.int64(2): {'Billy Joe'},
 np.int64(3): {'Billy Joe'},
 np.int64(4): {'Billy Joe'},
 np.int64(5): {'Billy Joe'},
 np.int64(6): {'Billy Joe'},
 np.int64(7): {'Billy Joe'},
 np.int64(8): {'Carlos Cabrera'},
 np.int64(9): {'Carlos Cabrera'},
 np.int64(10): {'Carlos Cabrera'},
 np.int64(11): {'Carlos Cabrera'},
 np.int64(12): {'Carlos Cabrera'},
 np.int64(13): {'Carlos Cabrera'},
 np.int64(14): {'Carlos Cabrera'},
 np.int64(15): {'Carlos Cabrera'},
 np.int64(16): {'Carlos Cabrera'},
 np.int64(17): {'Carlos Cabrera'},
 np.int64(18): {'Carlos Cabrera'},
 np.int64(19): {'DB'},
 np.int64(20): {'DB'},
 np.int64(21): {'DB'},
 np.int64(22): {'DB'},
 np.int64(23): {'DB'},
 np.int64(24): {'DB'},
 np.int64(25): {'DB'},
 np.int64(26): {'DB'},
 np.int64(27): {'DB'},
 np.int64(28): {'DB'},
 np.int64(29): {'DB'},
 np.int64(30): {'DB'},
 np.int64(31): {'DB'},
 np.int64(32): {'DB'},
 np.int64(33): {'DB'},
 np.int64(34): {'DB'},
 np.int64(35): {'

In [59]:
author_dict = {'0': {'Billy Joe'},
 '1': {'Billy Joe'},
 '2': {'Billy Joe'},
 '3': {'Billy Joe'},
 '4': {'Billy Joe'},
 '5': {'Billy Joe'},
 '6': {'Billy Joe'},
 '7': {'Billy Joe'},
 '8': {'Carlos Cabrera'},
 '9': {'Carlos Cabrera'},
 '10': {'Carlos Cabrera'},
 '11': {'Carlos Cabrera'},
 '12': {'Carlos Cabrera'},
 '13': {'Carlos Cabrera'},
 '14': {'Carlos Cabrera'},
 '15': {'Carlos Cabrera'},
 '16': {'Carlos Cabrera'},
 '17': {'Carlos Cabrera'},
 '18': {'Carlos Cabrera'},
 '19': {'DB'},
 '20': {'DB'},
 '21': {'DB'},
 '22': {'DB'},
 '23': {'DB'},
 '24': {'DB'},
 '25': {'DB'},
 '26': {'DB'},
 '27': {'DB'},
 '28': {'DB'},
 '29': {'DB'},
 '30': {'DB'},
 '31': {'DB'},
 '32': {'DB'},
 '33': {'DB'},
 '34': {'DB'},
 '35': {'DB'},
 '36': {'DB'},
 '37': {'DB'},
 '38': {'DB'},
 '39': {'DB'},
 '40': {'DB'},
 '41': {'DB'},
 '42': {'DB'},
 '43': {'DB'},
 '44': {'DB'},
 '45': {'DB'},
 '46': {'DB'},
 '47': {'DB'},
 '48': {'DB'},
 '49': {'DB'},
 '50': {'DB'},
 '51': {'DB'},
 '52': {'DB'},
 '53': {'DB'},
 '54': {'DB'},
 '55': {'DB'},
 '57': {'DB'},
 '58': {'DB'},
 '59': {'DB'},
 '60': {'DB'},
 '61': {'DB'},
 '62': {'DB'},
 '63': {'DB'},
 '64': {'DB'},
 '65': {'DB'},
 '66': {'DB'},
 '67': {'DB'},
 '68': {'DB'},
 '69': {'DB'},
 '70': {'DB'},
 '71': {'DB'},
 '72': {'DB'},
 '73': {'DB'},
 '74': {'DB'},
 '75': {'DB'},
 '76': {'DB'},
 '77': {'DB'},
 '78': {'DB'},
 '79': {'DB'},
 '80': {'DB'},
 '81': {'DB'},
 '82': {'DB'},
 '83': {'DB'},
 '85': {'DB'},
 '86': {'DB'},
 '87': {'DB'},
 '88': {'DB'},
 '89': {'DB'},
 '90': {'DB'},
 '91': {'DB'},
 '93': {'DB'},
 '94': {'DB'},
 '95': {'DB'},
 '96': {'DB'},
 '97': {'DB'},
 '98': {'DB'},
 '99': {'DB'},
 '100': {'DB'},
 '101': {'DB'},
 '102': {'DB'},
 '103': {'DB'},
 '104': {'DB'},
 '105': {'DB'},
 '106': {'DB'},
 '107': {'DB'},
 '108': {'DB'},
 '109': {'DB'},
 '110': {'DB'},
 '113': {'DB'},
 '114': {'DB'},
 '115': {'DB'},
 '116': {'DB'},
 '117': {'DB'},
 '118': {'DB'},
 '119': {'DB'},
 '120': {'DB'},
 '121': {'DB'},
 '122': {'DB'},
 '123': {'DB'},
 '124': {'DB'},
 '125': {'DB'},
 '126': {'DB'},
 '127': {'DB'},
 '128': {'DB'},
 '129': {'DB'},
 '130': {'DB'},
 '131': {'DB'},
 '132': {'DB'},
 '133': {'DB'},
 '134': {'DB'},
 '135': {'DB'},
 '136': {'DB'},
 '137': {'DB'},
 '138': {'DB'},
 '139': {'DB'},
 '140': {'DB'},
 '141': {'DB'},
 '144': {'DB'},
 '145': {'DB'},
 '146': {'DB'},
 '147': {'DB'},
 '148': {'DB'},
 '149': {'DB'},
 '150': {'DB'},
 '152': {'DB'},
 '153': {'DB'},
 '154': {'DB'},
 '155': {'DB'},
 '156': {'DB'},
 '157': {'DB'},
 '158': {'DB'},
 '160': {'DB'},
 '161': {'DB'},
 '162': {'DB'},
 '163': {'DB'},
 '164': {'DB'},
 '165': {'DB'},
 '166': {'DB'},
 '167': {'F C'},
 '168': {'F C'},
 '169': {'F C'},
 '170': {'F C'},
 '171': {'F C'},
 '172': {'F C'},
 '173': {'F C'},
 '174': {'F C'},
 '175': {'F C'},
 '176': {'F C'},
 '177': {'F C'},
 '178': {'F C'},
 '179': {'F C'},
 '180': {'F C'},
 '181': {'F C'},
 '182': {'F C'},
 '183': {'F C'},
 '184': {'F C'},
 '185': {'F C'},
 '186': {'F C'},
 '187': {'F C'},
 '188': {'F C'},
 '189': {'F C'},
 '190': {'F C'},
 '191': {'F C'},
 '192': {'F C'},
 '193': {'J H'},
 '194': {'J H'},
 '195': {'J H'},
 '196': {'J H'},
 '197': {'J H'},
 '198': {'Josh Mcclendon'},
 '199': {'Josh Mcclendon'},
 '200': {'Josh Mcclendon'},
 '201': {'Josh Mcclendon'},
 '202': {'Josh Mcclendon'},
 '203': {'Josh Mcclendon'},
 '205': {'Josh Mcclendon'},
 '206': {'Josh Mcclendon'},
 '207': {'Josh Mcclendon'},
 '209': {'Josh Mcclendon'},
 '210': {'Josh Mcclendon'},
 '211': {'Josh Mcclendon'},
 '212': {'Josh Mcclendon'},
 '213': {'Josh Mcclendon'},
 '214': {'Josh Mcclendon'},
 '215': {'Josh Mcclendon'},
 '216': {'Josh Mcclendon'},
 '217': {'Josh Mcclendon'},
 '218': {'Josh Mcclendon'},
 '219': {'Josh Mcclendon'},
 '220': {'Josh Mcclendon'},
 '221': {'Josh Mcclendon'},
 '222': {'Josh Mcclendon'},
 '223': {'Josh Mcclendon'},
 '224': {'Josh Mcclendon'},
 '225': {'Josh Mcclendon'},
 '226': {'Josh Mcclendon'},
 '227': {'Josh Mcclendon'},
 '228': {'Josh Mcclendon'},
 '229': {'Josh Mcclendon'},
 '230': {'Josh Mcclendon'},
 '231': {'Josh Mcclendon'},
 '232': {'Josh Mcclendon'},
 '233': {'Josh Mcclendon'},
 '235': {'Josh Mcclendon'},
 '236': {'Josh Mcclendon'},
 '237': {'Josh Mcclendon'},
 '238': {'Josh Mcclendon'},
 '239': {'Josh Mcclendon'},
 '240': {'Josh Mcclendon'},
 '241': {'Josh Mcclendon'},
 '242': {'Josh Mcclendon'},
 '243': {'Josh Mcclendon'},
 '244': {'Josh Mcclendon'},
 '245': {'Josh Mcclendon'},
 '246': {'Josh Mcclendon'},
 '247': {'Josh Mcclendon'},
 '248': {'Josh Mcclendon'},
 '249': {'Josh Mcclendon'},
 '250': {'Josh Mcclendon'},
 '251': {'Josh Mcclendon'},
 '252': {'Josh Mcclendon'},
 '253': {'Josh Mcclendon'},
 '254': {'Josh Mcclendon'},
 '255': {'Josh Mcclendon'},
 '256': {'Josh Mcclendon'},
 '257': {'Josh Mcclendon'},
 '258': {'Josh Mcclendon'},
 '259': {'Josh Mcclendon'},
 '260': {'Josh Mcclendon'},
 '261': {'Josh Mcclendon'},
 '262': {'Josh Mcclendon'},
 '263': {'Josh Mcclendon'},
 '264': {'Josh Mcclendon'},
 '265': {'Josh Mcclendon'},
 '266': {'Josh Mcclendon'},
 '267': {'Josh Mcclendon'},
 '268': {'Josh Mcclendon'},
 '269': {'Josh Mcclendon'},
 '270': {'Josh Mcclendon'},
 '271': {'Josh Mcclendon'},
 '272': {'Josh Mcclendon'},
 '273': {'Josh Mcclendon'},
 '274': {'Josh Mcclendon'},
 '275': {'Josh Mcclendon'},
 '276': {'Josh Mcclendon'},
 '277': {'Josh Mcclendon'},
 '278': {'Josh Mcclendon'},
 '279': {'Josh Mcclendon'},
 '280': {'Josh Mcclendon'},
 '281': {'Josh Mcclendon'},
 '282': {'Josh Mcclendon'},
 '284': {'Josh Mcclendon'},
 '285': {'Roger Cassidy'},
 '286': {'Roger Cassidy'},
 '287': {'Ryan Thompson'},
 '288': {'Ryan Thompson'},
 '289': {'Ryan Thompson'},
 '290': {'Ryan Thompson'},
 '291': {'Ryan Thompson'},
 '292': {'Ryan Thompson'},
 '293': {'Ryan Thompson'},
 '294': {'Ryan Thompson'},
 '295': {'Ryan Thompson'},
 '296': {'Ryan Thompson'},
 '297': {'Ryan Thompson'},
 '298': {'Ryan Thompson'},
 '299': {'Ryan Thompson'},
 '300': {'Ryan Thompson'},
 '301': {'Ryan Thompson'},
 '302': {'Ryan Thompson'},
 '303': {'Ryan Thompson'},
 '304': {'Ryan Thompson'},
 '305': {'allenriley2011'},
 '306': {'allenriley2011'},
 '308': {'allenriley2011'},
 '310': {'allenriley2011'},
 '311': {'allenriley2011'},
 '312': {'allenriley2011'},
 '313': {'allenriley2011'},
 '314': {'allenriley2011'},
 '315': {'allenriley2011'},
 '316': {'allenriley2011'},
 '317': {'allenriley2011'},
 '318': {'allenriley2011'},
 '319': {'allenriley2011'},
 '320': {'allenriley2011'},
 '321': {'allenriley2011'},
 '323': {'allenriley2011'},
 '324': {'allenriley2011'},
 '325': {'allenriley2011'},
 '326': {'allenriley2011'},
 '327': {'allenriley2011'},
 '328': {'allenriley2011'},
 '329': {'allenriley2011'},
 '330': {'allenriley2011'},
 '331': {'allenriley2011'},
 '332': {'allenriley2011'},
 '333': {'allenriley2011'},
 '334': {'allenriley2011'},
 '335': {'allenriley2011'},
 '336': {'allenriley2011'},
 '338': {'allenriley2011'},
 '339': {'allenriley2011'},
 '340': {'allenriley2011'},
 '341': {'allenriley2011'},
 '342': {'allenriley2011'},
 '343': {'allenriley2011'},
 '344': {'allenriley2011'},
 '345': {'allenriley2011'},
 '346': {'allenriley2011'},
 '347': {'allenriley2011'},
 '348': {'allenriley2011'},
 '349': {'allenriley2011'},
 '351': {'allenriley2011'},
 '352': {'allenriley2011'},
 '353': {'allenriley2011'},
 '354': {'allenriley2011'},
 '355': {'allenriley2011'},
 '356': {'allenriley2011'},
 '357': {'allenriley2011'},
 '358': {'allenriley2011'},
 '359': {'allenriley2011'},
 '360': {'bossofct'},
 '361': {'bossofct'},
 '365': {'chrisnr1998'},
 '366': {'chrisnr1998'},
 '367': {'chrisnr1998'},
 '368': {'chrisnr1998'},
 '371': {'chrisnr1998'},
 '372': {'chrisnr1998'},
 '373': {'chrisnr1998'},
 '374': {'chrisnr1998'},
 '375': {'chrisnr1998'},
 '377': {'chrisnr1998'},
 '379': {'chrisnr1998'},
 '380': {'chrisnr1998'},
 '381': {'chrisnr1998'},
 '382': {'chrisnr1998'},
 '383': {'chrisnr1998'},
 '384': {'chrisnr1998'},
 '385': {'chrisnr1998'},
 '388': {'chrisnr1998'},
 '389': {'chrisnr1998'},
 '390': {'chrisnr1998'},
 '391': {'chrisnr1998'},
 '392': {'chrisnr1998'},
 '395': {'chrisnr1998'},
 '396': {'chrisnr1998'},
 '397': {'chrisnr1998'},
 '398': {'chrisnr1998'},
 '399': {'chrisnr1998'},
 '400': {'chrisnr1998'},
 '401': {'chrisnr1998'},
 '402': {'chrisnr1998'},
 '403': {'chrisnr1998'},
 '404': {'chrisnr1998'},
 '405': {'chrisnr1998'},
 '406': {'chrisnr1998'},
 '407': {'chrisnr1998'},
 '408': {'chrisnr1998'},
 '409': {'chrisnr1998'},
 '410': {'chrisnr1998'},
 '411': {'chrisnr1998'},
 '412': {'chrisnr1998'},
 '415': {'chrisnr1998'},
 '416': {'chrisnr1998'},
 '417': {'chrisnr1998'},
 '418': {'chrisnr1998'},
 '419': {'chrisnr1998'},
 '420': {'chrisnr1998'},
 '423': {'chrisnr1998'},
 '424': {'chrisnr1998'},
 '425': {'chrisnr1998'},
 '429': {'chrisnr1998'},
 '430': {'chrisnr1998'},
 '431': {'chrisnr1998'},
 '432': {'chrisnr1998'},
 '433': {'chrisnr1998'},
 '434': {'chrisnr1998'},
 '435': {'christopher_brown1991'},
 '436': {'christopher_brown1991'},
 '437': {'christopher_brown1991'},
 '438': {'christopher_brown1991'},
 '440': {'christopher_brown1991'},
 '442': {'christopher_brown1991'},
 '443': {'christopher_brown1991'},
 '444': {'christopher_brown1991'},
 '445': {'christopher_brown1991'},
 '446': {'christopher_brown1991'},
 '448': {'christopher_brown1991'},
 '449': {'christopher_brown1991'},
 '450': {'christopher_brown1991'},
 '451': {'christopher_brown1991'},
 '453': {'christopher_brown1991'},
 '454': {'christopher_brown1991'},
 '456': {'christopher_brown1991'},
 '457': {'christopher_brown1991'},
 '458': {'christopher_brown1991'},
 '459': {'christopher_brown1991'},
 '460': {'christopher_brown1991'},
 '461': {'christopher_brown1991'},
 '462': {'christopher_brown1991'},
 '463': {'christopher_brown1991'},
 '467': {'christopher_brown1991'},
 '468': {'christopher_brown1991'},
 '469': {'christopher_brown1991'},
 '472': {'christopher_brown1991'},
 '473': {'christopher_brown1991'},
 '474': {'georgiamike1968'},
 '475': {'georgiamike1968'},
 '476': {'georgiamike1968'},
 '477': {'georgiamike1968'},
 '478': {'georgiamike1968'},
 '479': {'georgiamike1968'},
 '481': {'georgiamike1968'},
 '482': {'georgiamike1968'},
 '483': {'georgiamike1968'},
 '484': {'georgiamike1968'},
 '485': {'georgiamike1968'},
 '486': {'georgiamike1968'},
 '487': {'georgiamike1968'},
 '488': {'georgiamike1968'},
 '489': {'georgiamike1968'},
 '490': {'georgiamike1968'},
 '491': {'georgiamike1968'},
 '492': {'georgiamike1968'},
 '493': {'georgiamike1968'},
 '494': {'georgiamike1968'},
 '495': {'georgiamike1968'},
 '496': {'georgiamike1968'},
 '497': {'georgiamike1968'},
 '498': {'georgiamike1968'},
 '499': {'georgiamike1968'},
 '500': {'georgiamike1968'},
 '501': {'georgiamike1968'},
 '502': {'georgiamike1968'},
 '503': {'georgiamike1968'},
 '504': {'georgiamike1968'},
 '505': {'georgiamike1968'},
 '506': {'georgiamike1968'},
 '507': {'georgiamike1968'},
 '508': {'georgiamike1968'},
 '509': {'georgiamike1968'},
 '511': {'georgiamike1968'},
 '512': {'georgiamike1968'},
 '513': {'georgiamike1968'},
 '514': {'georgiamike1968'},
 '515': {'georgiamike1968'},
 '516': {'georgiamike1968'},
 '517': {'georgiamike1968'},
 '518': {'georgiamike1968'},
 '519': {'georgiamike1968'},
 '520': {'georgiamike1968'},
 '522': {'georgiamike1968'},
 '523': {'georgiamike1968'},
 '524': {'georgiamike1968'},
 '525': {'georgiamike1968'},
 '526': {'georgiamike1968'},
 '528': {'georgiamike1968'},
 '529': {'georgiamike1968'},
 '530': {'georgiamike1968'},
 '531': {'georgiamike1968'},
 '533': {'georgiamike1968'},
 '534': {'georgiamike1968'},
 '535': {'georgiamike1968'},
 '536': {'georgiamike1968'},
 '537': {'georgiamike1968'},
 '538': {'georgiamike1968'},
 '539': {'georgiamike1968'},
 '542': {'georgiamike1968'},
 '543': {'georgiamike1968'},
 '544': {'georgiamike1968'},
 '545': {'georgiamike1968'},
 '546': {'georgiamike1968'},
 '547': {'georgiamike1968'},
 '548': {'georgiamike1968'},
 '549': {'georgiamike1968'},
 '550': {'georgiamike1968'},
 '551': {'georgiamike1968'},
 '552': {'georgiamike1968'},
 '553': {'georgiamike1968'},
 '554': {'georgiamike1968'},
 '555': {'georgiamike1968'},
 '556': {'georgiamike1968'},
 '557': {'georgiamike1968'},
 '558': {'georgiamike1968'},
 '559': {'georgiamike1968'},
 '560': {'georgiamike1968'},
 '561': {'georgiamike1968'},
 '562': {'georgiamike1968'},
 '563': {'georgiamike1968'},
 '564': {'georgiamike1968'},
 '565': {'georgiamike1968'},
 '566': {'georgiamike1968'},
 '567': {'georgiamike1968'},
 '568': {'georgiamike1968'},
 '569': {'georgiamike1968'},
 '570': {'georgiamike1968'},
 '571': {'georgiamike1968'},
 '572': {'georgiamike1968'},
 '573': {'georgiamike1968'},
 '574': {'georgiamike1968'},
 '575': {'georgiamike1968'},
 '576': {'georgiamike1968'},
 '577': {'georgiamike1968'},
 '579': {'georgiamike1968'},
 '580': {'georgiamike1968'},
 '581': {'georgiamike1968'},
 '582': {'georgiamike1968'},
 '583': {'georgiamike1968'},
 '584': {'georgiamike1968'},
 '585': {'georgiamike1968'},
 '586': {'georgiamike1968'},
 '587': {'georgiamike1968'},
 '588': {'georgiamike1968'},
 '589': {'georgiamike1968'},
 '590': {'georgiamike1968'},
 '591': {'georgiamike1968'},
 '592': {'innocentz6197@sbcglobal.net'},
 '593': {'innocentz6197@sbcglobal.net'},
 '594': {'innocentz6197@sbcglobal.net'},
 '595': {'innocentz6197@sbcglobal.net'},
 '596': {'innocentz6197@sbcglobal.net'},
 '597': {'innocentz6197@sbcglobal.net'},
 '598': {'innocentz6197@sbcglobal.net'},
 '599': {'innocentz6197@sbcglobal.net'},
 '600': {'innocentz6197@sbcglobal.net'},
 '601': {'innocentz6197@sbcglobal.net'},
 '602': {'innocentz6197@sbcglobal.net'},
 '603': {'innocentz6197@sbcglobal.net'},
 '604': {'innocentz6197@sbcglobal.net'},
 '605': {'innocentz6197@sbcglobal.net'},
 '606': {'innocentz6197@sbcglobal.net'},
 '607': {'innocentz6197@sbcglobal.net'},
 '608': {'innocentz6197@sbcglobal.net'},
 '609': {'innocentz6197@sbcglobal.net'},
 '610': {'innocentz6197@sbcglobal.net'},
 '611': {'innocentz6197@sbcglobal.net'},
 '612': {'innocentz6197@sbcglobal.net'},
 '613': {'innocentz6197@sbcglobal.net'},
 '614': {'innocentz6197@sbcglobal.net'},
 '615': {'innocentz6197@sbcglobal.net'},
 '617': {'innocentz6197@sbcglobal.net'},
 '618': {'innocentz6197@sbcglobal.net'},
 '619': {'jackjohnsons7'},
 '620': {'jackjohnsons7'},
 '621': {'jackjohnsons7'}}
author_dict

{'0': {'Billy Joe'},
 '1': {'Billy Joe'},
 '2': {'Billy Joe'},
 '3': {'Billy Joe'},
 '4': {'Billy Joe'},
 '5': {'Billy Joe'},
 '6': {'Billy Joe'},
 '7': {'Billy Joe'},
 '8': {'Carlos Cabrera'},
 '9': {'Carlos Cabrera'},
 '10': {'Carlos Cabrera'},
 '11': {'Carlos Cabrera'},
 '12': {'Carlos Cabrera'},
 '13': {'Carlos Cabrera'},
 '14': {'Carlos Cabrera'},
 '15': {'Carlos Cabrera'},
 '16': {'Carlos Cabrera'},
 '17': {'Carlos Cabrera'},
 '18': {'Carlos Cabrera'},
 '19': {'DB'},
 '20': {'DB'},
 '21': {'DB'},
 '22': {'DB'},
 '23': {'DB'},
 '24': {'DB'},
 '25': {'DB'},
 '26': {'DB'},
 '27': {'DB'},
 '28': {'DB'},
 '29': {'DB'},
 '30': {'DB'},
 '31': {'DB'},
 '32': {'DB'},
 '33': {'DB'},
 '34': {'DB'},
 '35': {'DB'},
 '36': {'DB'},
 '37': {'DB'},
 '38': {'DB'},
 '39': {'DB'},
 '40': {'DB'},
 '41': {'DB'},
 '42': {'DB'},
 '43': {'DB'},
 '44': {'DB'},
 '45': {'DB'},
 '46': {'DB'},
 '47': {'DB'},
 '48': {'DB'},
 '49': {'DB'},
 '50': {'DB'},
 '51': {'DB'},
 '52': {'DB'},
 '53': {'DB'},
 '54': {'DB'

In [60]:
# message_idx을 str로 변환
pjz_df['message_idx'] = pjz_df['message_idx'].astype(str)
pjz_df['author'] = pjz_df['message_idx'].map(author_dict)
pjz_df['author'] = pjz_df['author'].apply(lambda x: list(x)[0])
pjz_df

,source,label,message_idx,author,text
0,PJ chats,1,0,Billy Joe,"안녕, 나 매즈야"
1,PJ chats,1,0,Billy Joe,"안녕, 자기야"
2,PJ chats,1,0,Billy Joe,거의 일 다 끝났어. 정말 기뻐
3,PJ chats,1,0,Billy Joe,안녕 :) 그거 좋은 거지?
4,PJ chats,1,0,Billy Joe,응
...,...,...,...,...,...
7569,PJ chats,1,621,jackjohnsons7,"나 곧 가야 해, 엄마가 소리치고 있어."
7570,PJ chats,1,621,jackjohnsons7,그거 좋았어?
7571,PJ chats,1,621,jackjohnsons7,그래서 너 하트포드 어디 거리에서 살아?
7572,PJ chats,1,621,jackjohnsons7,너한테 안 알려줘.


In [61]:
pjz_df.groupby('author')['text'].count()

author
Billy Joe                       551
Carlos Cabrera                  379
DB                             1570
F C                             355
J H                             155
Josh Mcclendon                  537
Roger Cassidy                   106
Ryan Thompson                   271
allenriley2011                  618
bossofct                        117
chrisnr1998                     990
christopher_brown1991           572
georgiamike1968                 776
innocentz6197@sbcglobal.net     427
jackjohnsons7                   150
Name: text, dtype: int64

In [62]:
# 같은 author를 가진 데이터를 하나의 row로 합치기
pjz_df['text'] = pjz_df['text'].astype(str)
pjz_df = pjz_df.groupby('author')['text'].apply(lambda x: '\t'.join(x)).reset_index()
pjz_df

,author,text
0,Billy Joe,"안녕, 나 매즈야\t안녕, 자기야\t거의 일 다 끝났어. 정말 기뻐\t안녕 :) 그..."
1,Carlos Cabrera,알겠어 안녕\t안녕 이거 카를로스야?\t응 나야\t섹시해\t^_^\t뭐해 베이비\t...
2,DB,안녕 DB!:)\tㅋㅋ 어떻게 지내\t난 좋아;) 너는 어때?\t음 그렇겠지 ㅋㅋ ...
3,F C,안녕 공주\t네 번호를 폰에 저장하고 있었어.. 근데 이름이 뭐야??\t하하. 응 ...
4,J H,안녕?\t미트미에서 온 남자야 ㅋㅋ\t이름이 뭐야??\t제레미\t집에 혼자 있어? ...
5,Josh Mcclendon,"안녕 미트미의 조쉬야\t그래서, 너는 지금까지 어떤 성적인 경험을 해봤는지 궁금해...."
6,Roger Cassidy,안녕 로저야\t수업 중이야\t17살이라고 했잖아.. 그게 왜 나쁘지?\t아니 나쁘지...
7,Ryan Thompson,이거 라이언이야??\t별거 없어 집에서 쉬고 있어\t놀러 갈래\t뭐 진짜로?\t왜 ...
8,allenriley2011,앨런이야 뭐해\t침대에 있는데 아무것도 안 해 껴안고 싶어\t히히.. 나도. 그래\...
9,bossofct,나야 셰인\t안녕ㅎㅎㅎ\t평소처럼 심심해 ㅋㅋ 너는?\t말로니에서 얼마나 멀어?\t...


In [63]:
# author별로 대화 내용을 리스트로 저장
def split_text(df, x, author):
    return df[df[x] == author]['text'].values[0].split('\t')

In [64]:
# pjz_df 데이터셋을 train, valid, test로 나누기
# Valid : jackjohnsons7, allenriley2011
# Test : f_c, innocentz6197_sbcglobal_net
# Train : 나머지

pjz_train = pjz_df[pjz_df['author'].isin(['Billy Joe', 'Carlos Cabrera', 'DB', 'J H', 'Josh Mcclendon', 'Roger Cassidy', 'Ryan Thompson', 'bossofct', 'chrisnr1998', 'christopher_brown1991', 'georgiamike1968'])]
pjz_valid = pjz_df[pjz_df['author'].isin(['jackjohnsons7', 'allenriley2011'])]
pjz_test = pjz_df[pjz_df['author'].isin(['F C', 'innocentz6197@sbcglobal.net'])]

pjz_train

,author,text
0,Billy Joe,"안녕, 나 매즈야\t안녕, 자기야\t거의 일 다 끝났어. 정말 기뻐\t안녕 :) 그..."
1,Carlos Cabrera,알겠어 안녕\t안녕 이거 카를로스야?\t응 나야\t섹시해\t^_^\t뭐해 베이비\t...
2,DB,안녕 DB!:)\tㅋㅋ 어떻게 지내\t난 좋아;) 너는 어때?\t음 그렇겠지 ㅋㅋ ...
4,J H,안녕?\t미트미에서 온 남자야 ㅋㅋ\t이름이 뭐야??\t제레미\t집에 혼자 있어? ...
5,Josh Mcclendon,"안녕 미트미의 조쉬야\t그래서, 너는 지금까지 어떤 성적인 경험을 해봤는지 궁금해...."
6,Roger Cassidy,안녕 로저야\t수업 중이야\t17살이라고 했잖아.. 그게 왜 나쁘지?\t아니 나쁘지...
7,Ryan Thompson,이거 라이언이야??\t별거 없어 집에서 쉬고 있어\t놀러 갈래\t뭐 진짜로?\t왜 ...
9,bossofct,나야 셰인\t안녕ㅎㅎㅎ\t평소처럼 심심해 ㅋㅋ 너는?\t말로니에서 얼마나 멀어?\t...
10,chrisnr1998,별로 없어. 너는?\t방금 저녁 먹었어\t뭐 먹었어?\t치킨\tㅋㅋ 맛있겠다. 나도...
11,christopher_brown1991,"안녕.\t안녕~~~~~!\t뭐 해, 누군지 알아?\t아무것도 안 해~~~ 그리고 응..."


In [65]:
# Sliding Window('\t' ver.)
def sliding_window(texts, window_size=30, stride=1):
    windows = []
    if len(texts) < window_size:
        text = "[CLS] " + " ".join(str(m) + " [SEP]" for m in texts)
        windows.append(text)
    else:
        for i in range(0, len(texts)-window_size+1, stride):
            text = "[CLS] " + " ".join(str(m) + " [SEP]" for m in texts[i:i+window_size])
            windows.append(text)
    return windows

In [66]:
# grooming Train, Valid, Test 데이터셋 생성
train_texts = []
valid_texts = []
test_texts = []

for author in pjz_train['author']:
    train_texts.extend(sliding_window(split_text(pjz_train, 'author', author)))
for author in pjz_valid['author']:
    valid_texts.extend(sliding_window(split_text(pjz_valid, 'author', author)))
for author in pjz_test['author']:
    test_texts.extend(sliding_window(split_text(pjz_test, 'author', author)))
    
pjz_train = pd.DataFrame({'text': train_texts, 'label': 1})
pjz_valid = pd.DataFrame({'text': valid_texts, 'label': 1})
pjz_test = pd.DataFrame({'text': test_texts, 'label': 1})
    
print('PJZ Train :', pjz_train.shape)
print('PJZ Valid :', pjz_valid.shape)
print('PJZ Test :', pjz_test.shape)

PJZ Train : (5705, 2)
PJZ Valid : (710, 2)
PJZ Test : (724, 2)


# AI Hub Dataset

In [67]:
def convert_chat_df(source, start, end):
    annotations_subject = []
    message_idx = []
    author = []
    text = []

    for i in range(start, end):
        for j in range(1, 21):
            if j < 10:
                with open(f'/home/k-cat/TH/K-CAT/lth/data/AIHub/{source}/{source.upper()}_{i}_0{j}.json') as f:
                    source_data = json.load(f)
            else:
                with open(f'/home/k-cat/TH/K-CAT/lth/data/AIHub/{source}/{source.upper()}_{i}_{j}.json') as f:
                    source_data = json.load(f)  
            
            if source_data['info'][0]['annotations']['speaker_type'] == '1:1':
                annotations_subject.append(source_data['info'][0]['annotations']['subject'])
                for t in source_data['info'][0]['annotations']['text'].split('\n'):
                    split_idx = t.find(':')
                    n, s = t[:split_idx], t[split_idx+1:]
                    message_idx.append(f'{i}_{j}')
                    author.append(n.strip())
                    text.append(s.strip())
                    
    df = pd.DataFrame({'source': source, 'label': 0, 'message_idx': message_idx, 'author': author, 'text': text})
    
    return df, Counter(annotations_subject)

In [68]:
kakao_df, k_category_cnt = convert_chat_df('kakao', 899, 909)
facebook_df, f_category_cnt = convert_chat_df('facebook', 101, 111)
instagram_df, i_category_cnt = convert_chat_df('instagram', 61, 71)
nateon_df, n_category_cnt = convert_chat_df('nateon', 22, 32)

In [69]:
all_categories = set(k_category_cnt.keys()) | set(f_category_cnt.keys()) | set(i_category_cnt.keys()) | set(n_category_cnt.keys())

category_data = []

for category in sorted(all_categories):
    category_data.append([
        category,
        k_category_cnt.get(category, 0),
        f_category_cnt.get(category, 0),
        i_category_cnt.get(category, 0),
        n_category_cnt.get(category, 0)
    ])

category_df = pd.DataFrame(category_data, columns=['Category', 'K Count', 'F Count', 'I Count', 'N Count'])
category_df

,Category,K Count,F Count,I Count,N Count
0,가족,8,10,10,5
1,건강,8,10,10,5
2,게임,8,10,10,5
3,계절/날씨,8,10,10,5
4,교육,8,10,11,5
5,교통,8,10,10,5
6,군대,8,10,10,5
7,미용,8,10,9,5
8,반려동물,8,10,10,5
9,방송/연예,8,10,10,5


In [70]:
aihub_df = pd.concat([kakao_df, facebook_df, instagram_df, nateon_df])
aihub_df

,source,label,message_idx,author,text
0,kakao,0,899_1,1,오빠야 저녁 식사는 뭐야?
1,kakao,0,899_1,2,나는 아직 뭐 먹을지 모르겠네 니는?
2,kakao,0,899_1,1,난 일단 지금 라면 먹고 있는데 저녁 모르겠어 키키
3,kakao,0,899_1,2,키키 오 지금 라면을 먹음 저녁 먹을 수 있나
4,kakao,0,899_1,1,모르겠어 키키 이제 일어나서 키키
...,...,...,...,...,...
3295,nateon,0,26_20,2,역시 좋은 애니에 좋은 음악까지...
3296,nateon,0,26_20,2,중요한 역할을 하는 거 같아!
3297,nateon,0,26_20,1,맞아! 하울의 움직이는 성도 너무 좋았어!
3298,nateon,0,26_20,2,헉 나 진짜 좋아하는 애니야.


In [71]:
aihub_df['text'] = aihub_df['text'].str.replace('키키', 'ㅋㅋ')
aihub_df

,source,label,message_idx,author,text
0,kakao,0,899_1,1,오빠야 저녁 식사는 뭐야?
1,kakao,0,899_1,2,나는 아직 뭐 먹을지 모르겠네 니는?
2,kakao,0,899_1,1,난 일단 지금 라면 먹고 있는데 저녁 모르겠어 ㅋㅋ
3,kakao,0,899_1,2,ㅋㅋ 오 지금 라면을 먹음 저녁 먹을 수 있나
4,kakao,0,899_1,1,모르겠어 ㅋㅋ 이제 일어나서 ㅋㅋ
...,...,...,...,...,...
3295,nateon,0,26_20,2,역시 좋은 애니에 좋은 음악까지...
3296,nateon,0,26_20,2,중요한 역할을 하는 거 같아!
3297,nateon,0,26_20,1,맞아! 하울의 움직이는 성도 너무 좋았어!
3298,nateon,0,26_20,2,헉 나 진짜 좋아하는 애니야.


In [72]:
# 같은 author를 가진 데이터를 하나의 row로 합치기
aihub_df = aihub_df.groupby('message_idx')['text'].apply(lambda x: '\t'.join(x)).reset_index()
aihub_df

,message_idx,text
0,101_1,언니 명랑 핫도그 먹어 봤어?\t웅 먹은 지 한참 됐어\t나 오늘 먹었는데 진짜 존...
1,101_10,언니 요즘 운동하고 있는 거 맞아?\t나 요새 운동 안 해\t살 엄청 찜\tㅋㅋ 근...
2,101_11,언니는 요즘에 핸드폰으로 게임 안 해?\t나 안 해\t너 해?\t나 지금 한 면이 ...
3,101_12,언니 이번 년도 얼마 안 남았는데 가고 싶은 곳 있어?\t나는 진짜 해외 가고 싶다...
4,101_13,와 오늘 분명히 날씨 풀린다고 그랬는데?\t와 진짜 개추웠지\t패딩 꺼내자\tㅋㅋ ...
...,...,...
654,908_5,야 니 그거 봤나 ㅋㅋ 논산 주민들이 민원 넣은 거\t논산? 거기 육군 훈련소 있는...
655,908_6,야 요즘 애들은 피아노 학원 안 다니는 거 아나?\t진짜? 우리 때는 필수 사교육이...
656,908_7,아 우리 아빠는 맨날 늦게 들어옴\t진짜? 우리 아빠는 항상 칼퇴근 칼귀가하시는데\...
657,908_8,남자친구랑 또 싸웠음\t왜 이렇게 잔소리가 많지?\tㅋㅋ 니 남친 잔소리가 좀 심하...


In [73]:
# non_grooming 데이터셋을 train, valid, test로 나누기

aihub_train, aihub_valid = train_test_split(aihub_df, test_size=0.2, random_state=62)
aihub_valid, aihub_test = train_test_split(aihub_valid, test_size=0.5, random_state=62)

train_texts = []
valid_texts = []
test_texts = []

for idx in aihub_train['message_idx']:
    train_texts.extend(sliding_window(split_text(aihub_df, 'message_idx', idx)))
for idx in aihub_valid['message_idx']:
    valid_texts.extend(sliding_window(split_text(aihub_df, 'message_idx', idx)))
for idx in aihub_test['message_idx']:
    test_texts.extend(sliding_window(split_text(aihub_df, 'message_idx', idx)))

aihub_train = pd.DataFrame({'text': train_texts, 'label': 0})
aihub_valid = pd.DataFrame({'text': valid_texts, 'label': 0})
aihub_test = pd.DataFrame({'text': test_texts, 'label': 0})

print('AIHub Train :', aihub_train.shape)
print('AIHub Valid :', aihub_valid.shape)
print('AIHub Test :', aihub_test.shape)

AIHub Train : (898, 2)
AIHub Valid : (98, 2)
AIHub Test : (87, 2)


# SBS Dataset

In [74]:
sbs_df = pd.read_excel('/home/k-cat/TH/K-CAT/lth/data/SBS/SBS.xlsx')

In [75]:
# 대화 내용 합치기
for i in range(len(sbs_df)):
    chat = []
    for j in range(11, 95):
        content = sbs_df.iloc[i, j]
        if pd.notna(content):
            chat.append(content)
        else:
            break
    sbs_df['대화'][i] = chat

sbs_df.drop(sbs_df.columns[12:95], axis=1, inplace=True)

In [76]:
# 필요한 데이터만 남기기
sbs_df.rename(columns={'Unnamed: 0': 'message_idx', '어플': 'source', '제안내용': 'label', '대화': 'text'}, inplace=True)
sbs_df = sbs_df[['source', 'label', 'message_idx', 'text']]

In [77]:
sbs_df

,source,label,message_idx,text
0,앙챗,음란채팅,chat0001,"[하이, 아침부터 꼴리네요, ㅎㅇ, 아침부터 꼴려서, 고추나 만지고 잇어요ㅎ, 네;..."
1,앙챗,음란채팅,chat0002,"[저기요 시골 학교 뒤에서 밤에 여학생 따먹은얘기 라인으로 듣기만 할래요?, ㅎㅎ;..."
2,앙챗,음란채팅,chat0003,"[ㅎㅇ, ㅎㅇ, 네 반가워요, 저도 반가워요, 야한얘기?, 다른 얘기.., 어떤?,..."
3,앙챗,단순채팅,chat0004,"[아조씨랑 아무얘기ㄱㄱ, 안녕, 하이하이, 오늘 학교 안가심?, 말이 없누…학교갔나..."
4,앙챗,성매수,chat0005,"[조건, 보실분 구해요, 저 학생이에요, 괜찮아요, 학교 다녀요?, ?, 네 학교 ..."
...,...,...,...,...
1029,동네친구,성매수,chat1030,"[님요. 시간되나여., 안녕하세요., 넹. 만남인가여., 무슨말씀, 조건? 친구?,..."
1030,동네친구,단순채팅,chat1031,"[좋은하루되세요, 안녕하세요., 네네]"
1031,동네친구,단순채팅,chat1032,[안녕하세요 처음이용하는거라 괜찮으면 카톡으로 해요. Win221 친추해주세요~ 친...
1032,동네친구,성매수,chat1033,"[안녕하세요??, 안녕하세요~, ㅈㄱ인가요?, 정확하게 말씀해주세요, 조건인가요?,..."


In [78]:
sbs_df.to_csv('/home/k-cat/TH/K-CAT/lth/data/SBS/SBS.csv', index=False)

In [79]:
sbs_df['text'] = sbs_df['text'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
type(sbs_df['text'][0])

list

In [80]:
sbs_df['label'].value_counts()

label
음란채팅    330
단순채팅    313
성매수     265
음란만남     64
단순만남     59
기타        3
Name: count, dtype: int64

In [81]:
# 채팅 내용 개수
sbs_df['text_len'] = sbs_df['text'].apply(lambda x: len(x))
sbs_df['text_len'].describe()

count    1034.000000
mean       14.519342
std        11.692747
min         1.000000
25%         7.000000
50%        12.000000
75%        19.000000
max        84.000000
Name: text_len, dtype: float64

In [82]:
# 대화 내용이 3개 이하 제거
sbs_df = sbs_df[sbs_df['text_len'] > 3]
sbs_df

,source,label,message_idx,text,text_len
0,앙챗,음란채팅,chat0001,"[하이, 아침부터 꼴리네요, ㅎㅇ, 아침부터 꼴려서, 고추나 만지고 잇어요ㅎ, 네;...",6
2,앙챗,음란채팅,chat0003,"[ㅎㅇ, ㅎㅇ, 네 반가워요, 저도 반가워요, 야한얘기?, 다른 얘기.., 어떤?,...",8
3,앙챗,단순채팅,chat0004,"[아조씨랑 아무얘기ㄱㄱ, 안녕, 하이하이, 오늘 학교 안가심?, 말이 없누…학교갔나...",17
4,앙챗,성매수,chat0005,"[조건, 보실분 구해요, 저 학생이에요, 괜찮아요, 학교 다녀요?, ?, 네 학교 ...",21
5,앙챗,성매수,chat0006,"[두번 30에 대나여? 소개부탁드립니다, 저 학생인데여;;, 만남 안하시나영.?, ...",4
...,...,...,...,...,...
1025,동네친구,단순채팅,chat1026,"[안녕?? 어떤 성격이야?, 왜요, 마음통하는 사람 찾는다길래 ㅋ, 아 저 16살인...",33
1026,동네친구,성매수,chat1027,"[마음? 몸?, 안녕하세요 무슨 말씀이신지 ㅎ??, 몸 맞는 사람은 안찾아?, 저 ...",37
1029,동네친구,성매수,chat1030,"[님요. 시간되나여., 안녕하세요., 넹. 만남인가여., 무슨말씀, 조건? 친구?,...",32
1032,동네친구,성매수,chat1033,"[안녕하세요??, 안녕하세요~, ㅈㄱ인가요?, 정확하게 말씀해주세요, 조건인가요?,...",35


In [83]:
### 채팅 검수를 위한 데이터 분리 ###
sbs_df.drop(['text_len'], axis=1, inplace=True)
sbs_df[sbs_df['label'] == '음란채팅'].to_csv('/home/k-cat/TH/K-CAT/lth/data/SBS/SBS_음란채팅.csv', index=False)
sbs_df[sbs_df['label'] == '단순채팅'].to_csv('/home/k-cat/TH/K-CAT/lth/data/SBS/SBS_단순채팅.csv', index=False) # 그루밍과 관련 없음 -> 일상 대화
sbs_df[sbs_df['label'] == '성매수'].to_csv('/home/k-cat/TH/K-CAT/lth/data/SBS/SBS_성매수.csv', index=False)
sbs_df[sbs_df['label'] == '음란만남'].to_csv('/home/k-cat/TH/K-CAT/lth/data/SBS/SBS_음란만남.csv', index=False)
sbs_df[sbs_df['label'] == '단순만남'].to_csv('/home/k-cat/TH/K-CAT/lth/data/SBS/SBS_단순만남.csv', index=False) # 그루밍과 관련 없음 -> 일상 대화
sbs_df[sbs_df['label'] == '기타'].to_csv('/home/k-cat/TH/K-CAT/lth/data/SBS/SBS_기타.csv', index=False) # 그루밍과 관련 없음


### 검수 완료 후 데이터 합치기 ###
data_1 = pd.read_csv('/home/k-cat/TH/K-CAT/lth/data/SBS/SBS_음란채팅.csv')
data_2 = pd.read_csv('/home/k-cat/TH/K-CAT/lth/data/SBS/SBS_단순채팅.csv')
data_3 = pd.read_csv('/home/k-cat/TH/K-CAT/lth/data/SBS/SBS_성매수.csv')
data_4 = pd.read_csv('/home/k-cat/TH/K-CAT/lth/data/SBS/SBS_음란만남.csv')
data_5 = pd.read_csv('/home/k-cat/TH/K-CAT/lth/data/SBS/SBS_단순만남.csv')

sbs_grooming_df = pd.concat([data_1, data_3, data_4], ignore_index=True)
sbs_non_grooming_df = pd.concat([data_2, data_5], ignore_index=True)

In [84]:
sbs_grooming_df['label'].value_counts()

label
음란채팅    305
성매수     260
음란만남     63
Name: count, dtype: int64

In [85]:
sbs_non_grooming_df['label'].value_counts()

label
단순채팅    253
단순만남     58
Name: count, dtype: int64

In [86]:
sbs_grooming_df.to_csv('/home/k-cat/TH/K-CAT/lth/data/SBS_grooming.csv', index=False)
sbs_non_grooming_df.to_csv('/home/k-cat/TH/K-CAT/lth/data/SBS_non_grooming.csv', index=False)

In [87]:
sbs_grooming = pd.read_csv('/home/k-cat/TH/K-CAT/lth/data/SBS_grooming.csv')
sbs_non_grooming = pd.read_csv('/home/k-cat/TH/K-CAT/lth/data/SBS_non_grooming.csv')

In [88]:
sbs_grooming['text'] = sbs_grooming['text'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
sbs_non_grooming['text'] = sbs_non_grooming['text'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [89]:
sbs_grooming_train, sbs_grooming_valid = train_test_split(sbs_grooming, test_size=0.2, random_state=50)
sbs_grooming_valid, sbs_grooming_test = train_test_split(sbs_grooming_valid, test_size=0.5, random_state=50)

non_grooming_train, non_grooming_valid = train_test_split(sbs_non_grooming, test_size=0.2, random_state=42)
non_grooming_valid, non_grooming_test = train_test_split(non_grooming_valid, test_size=0.5, random_state=42)

print('Grooming Chat Num :', len(sbs_grooming_train), len(sbs_grooming_valid), len(sbs_grooming_test))
print('Non-Grooming Chat Num :', len(non_grooming_train), len(non_grooming_valid), len(non_grooming_test))

Grooming Chat Num : 502 63 63
Non-Grooming Chat Num : 248 31 32


In [90]:
grooming_train_texts = []
grooming_valid_texts = []
grooming_test_texts = []

non_grooming_train_texts = []
non_grooming_valid_texts = []
non_grooming_test_texts = []

for i in range(len(sbs_grooming_train)):
    grooming_train_texts.extend(sliding_window(sbs_grooming_train['text'].iloc[i]))
for i in range(len(sbs_grooming_valid)):
    grooming_valid_texts.extend(sliding_window(sbs_grooming_valid['text'].iloc[i]))
for i in range(len(sbs_grooming_test)):
    grooming_test_texts.extend(sliding_window(sbs_grooming_test['text'].iloc[i]))
    
sbs_grooming_train = pd.DataFrame({'text': grooming_train_texts, 'label': 1})
sbs_grooming_valid = pd.DataFrame({'text': grooming_valid_texts, 'label': 1})
sbs_grooming_test = pd.DataFrame({'text': grooming_test_texts, 'label': 1})

for i in range(len(non_grooming_train)):
    non_grooming_train_texts.extend(sliding_window(non_grooming_train['text'].iloc[i]))
for i in range(len(non_grooming_valid)):
    non_grooming_valid_texts.extend(sliding_window(non_grooming_valid['text'].iloc[i]))
for i in range(len(non_grooming_test)):
    non_grooming_test_texts.extend(sliding_window(non_grooming_test['text'].iloc[i]))

sbs_non_grooming_train = pd.DataFrame({'text': non_grooming_train_texts, 'label': 0})
sbs_non_grooming_valid = pd.DataFrame({'text': non_grooming_valid_texts, 'label': 0})
sbs_non_grooming_test = pd.DataFrame({'text': non_grooming_test_texts, 'label': 0})

print('Grooming Train :', sbs_grooming_train.shape)
print('Grooming Valid :', sbs_grooming_valid.shape)
print('Grooming Test :', sbs_grooming_test.shape)
print()
print('Non-Grooming Train :', sbs_non_grooming_train.shape)
print('Non-Grooming Valid :', sbs_non_grooming_valid.shape)
print('Non-Grooming Test :', sbs_non_grooming_test.shape)

Grooming Train : (1052, 2)
Grooming Valid : (153, 2)
Grooming Test : (146, 2)

Non-Grooming Train : (595, 2)
Non-Grooming Valid : (73, 2)
Non-Grooming Test : (67, 2)


In [91]:
train_data = pd.concat([pjz_train, aihub_train, sbs_grooming_train, sbs_non_grooming_train])
valid_data = pd.concat([pjz_valid, aihub_valid, sbs_grooming_valid, sbs_non_grooming_valid])
test_data = pd.concat([pjz_test, aihub_test, sbs_grooming_test, sbs_non_grooming_test])

print(len(train_data), len(valid_data), len(test_data))

8250 1034 1024


# Final Dataset

In [ ]:
train_data.to_csv('/home/k-cat/TH/K-CAT/lth/data/training/train_data.csv', index=False)
valid_data.to_csv('/home/k-cat/TH/K-CAT/lth/data/training/valid_data.csv', index=False)
test_data.to_csv('/home/k-cat/TH/K-CAT/lth/data/training/test_data.csv', index=False)

In [5]:
train_data = pd.read_csv('/home/k-cat/TH/K-CAT/lth/data/training/train_data.csv')
valid_data = pd.read_csv('/home/k-cat/TH/K-CAT/lth/data/training/valid_data.csv')
test_data = pd.read_csv('/home/k-cat/TH/K-CAT/lth/data/training/test_data.csv')

train_data

,text,label
0,"[CLS] 안녕, 나 매즈야 [SEP] 안녕, 자기야 [SEP] 거의 일 다 끝났어...",1
1,"[CLS] 안녕, 자기야 [SEP] 거의 일 다 끝났어. 정말 기뻐 [SEP] 안녕...",1
2,[CLS] 거의 일 다 끝났어. 정말 기뻐 [SEP] 안녕 :) 그거 좋은 거지? ...,1
3,[CLS] 안녕 :) 그거 좋은 거지? [SEP] 응 [SEP] 그럼 뭐 새로운 일...,1
4,[CLS] 응 [SEP] 그럼 뭐 새로운 일 있어? ㅋㅋ [SEP] 없어 [SEP]...,1
...,...,...
8245,[CLS] ㅎㅎ안녕 나도 괜찮을까 나이를 떠나서 친구저럼 얘기 나누고 놀면 돼잖아^...,0
8246,"[CLS] ㅎㅇ [SEP] 몸에 자신있어하는 분이네 [SEP] 앗, 아까그분, 제가...",0
8247,[CLS] 하이루 [SEP] 하이요 [SEP] 학고에요? [SEP] 네 학교에요 [...,0
8248,[CLS] 어디세욤 [SEP] 저 서울이요 [SEP] 페이랑 프로필 부탁드려요 [S...,0
